# Reducing data-cubes using geometry objects

In [ ]:
# If first time running, uncomment the line below to install any additional dependencies
# !bash requirements-for-notebooks.sh

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from earthkit import transforms as ekt
from earthkit import data as ekd

from earthkit.data.testing import earthkit_remote_test_data_file

## Load some test data

All `earthkit-transforms` methods can be called with `earthkit-data` objects (Readers and Wrappers) or with the 
pre-loaded `xarray` or `geopandas` objects.

In this example we will use hourly ERA5 2m temperature data on a 0.5x0.5 spatial grid for the year 2015 as
our physical data; and we will use the NUTS geometries which are stored in a geojson file.

First we lazily load the ERA5 data  and NUTS geometries from our test-data repository.

Note the data is only downloaded when
we use it, e.g. at the `.to_xarray` line, additionally, the download is cached so the next time you run this
cell you will not need to re-download the file (unless it has been a very long time since you have run the
code, please see tutorials in `earthkit-data` for more details in cache management).

In [ ]:
# Get some demonstration ERA5 data, this could be any url or path to an ERA5 grib or netCDF file.
# remote_era5_file = earthkit_remote_test_data_file("era5_temperature_europe_2015.grib") # Large file
remote_era5_file = earthkit_remote_test_data_file("era5_temperature_europe_20150101.grib")
era5_data = ekd.from_source("url", remote_era5_file)
era5_xr = era5_data.to_xarray(time_dim_mode="valid_time").rename({"2t": "t2m"})
era5_xr

In [ ]:
# Use some demonstration polygons stored, this could be any url or path to geojson file
remote_nuts_url = earthkit_remote_test_data_file("NUTS_RG_60M_2021_4326_LEVL_0.geojson")
nuts_data = ekd.from_source("url", remote_nuts_url)

nuts_data.to_pandas()[:5]

## Reduce data
### Default behaviour

The default behaviour is to reduce the data along the spatial dimensions, only, and return the reduced data
in the Xarray format it was provided, i.e. `xr.DataArray` or `xr.Dataset`.

The returned object has a new dimension `FID` (feature id) which has a coordinate variable with the values
of the `FID` column in the input `geodataframe`.

The new variable name is made up of the original variable name and the method used to reduce, e.g. `t2m_mean`

In [ ]:
reduced_data = ekt.spatial.reduce(era5_xr, nuts_data)
reduced_data

### Reduce along additional dimension

For example, any time dimension, this is advisable as it ensures correct handling missing values and weights.

The extra_reduce_dims argument takes a single string or a list of strings corresponding to dimensions to
include in the reduction.

It is also possible to select a column in the geodataframe to use to populate the dimension and coordinate 
variable created by the reduction using the `mask_dim` kwarg, here we choose the `"FID"` column.

In [ ]:
reduced_data = ekt.spatial.reduce(
    era5_xr, nuts_data,
    mask_dim="FID", extra_reduce_dims='valid_time', all_touched=True
)
reduced_data

### Weighted reduction

Provide numpy/xarray arrays of weights, or use predefined weights options, i.e. `latitude`:

In [ ]:
reduced_data_xr = ekt.spatial.reduce(
    era5_xr, nuts_data, weights='latitude', mask_dim="FID", extra_reduce_dims='valid_time',
    all_touched=True
)
reduced_data_xr

## Return as a pandas dataframe

**WARNING: Returning reduced data in pandas format is considered experimental and may change in futureversions of earthkit** 

It is possible to return the reduced data in a fully expanded geopandas dataframe which contains the geometry 
and aggregated data.
Additional columns for the data values and rows and indexes added to fully describe the reduced data.

The returned object fully supports pandas indexing and in-built convenience methods (e.g. plotting),
but it comes with memory usage cost, hence in this example we reduce along all dimensions.

In [ ]:
reduced_data_pd = ekt.spatial.reduce(era5_xr, nuts_data, return_as="pandas", mask_dim="FID", extra_reduce_dims="valid_time")
reduced_data_pd[:10]

In [ ]:
reduced_data_pd.plot("t2m")
print('# Note that the NUTS regions include French foreign territories, hence the extent of the figure.')

## Appendix

### Unadvised: return_as = 'pandas' for time-series

This results in very heavy memory usage but may be useful

In [ ]:
reduced_data_pd = ekt.spatial.reduce(era5_xr, nuts_data, return_as="pandas", mask_dim="FID")
reduced_data_pd[:5]

In [ ]:
feature_index = "FID"
plot_var = "t2m"
# plot_x_vals = reduced_data.attrs[f"{plot_var}_dims"]["time"]
fig, ax = plt.subplots(1)
for feature in reduced_data_pd.index.get_level_values(feature_index).unique()[:5]:
    temp = reduced_data_pd.xs(feature, level=feature_index)
    temp[plot_var].plot(ax=ax, label=feature)
fig.legend()

## Providing a bespoke function for the reduction

When providing a our own function to the reduce method it must conform the the Xarray requirements defined in their [documentation pages](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.reduce.html#xarray-dataarray-reduce). Specifically:

*"Function which can be called in the form f(x, axis=axis, \*\*kwargs) to return the result of reducing an np.ndarray over an integer valued axis."*

Here we will calculate the ratio of the standard deviation to the mean for demonstration purposes only. We use nanmean and nanstd so that we ignore the nan values.

In [ ]:
def std_mean_ratio(x, axis=0, **kwargs):
    return np.nanstd(x, axis=axis, **kwargs) / np.nanmean(x, axis=axis, **kwargs)

In [ ]:
reduced_data = ekt.spatial.reduce(
    era5_xr, nuts_data,
    how = std_mean_ratio
)
reduced_data